# Tracking Models to Run Inferences on the MOT-16 dataset

Multi-Object Tracking (MOT) describes the task of identifying all objects in an image and following them over a sequence of frames. 

A wide range of Deep Learning-based MOT methods has been developed for human detection, but not for sparse/crowded space detection.

### 1. Install Packages and Setup Environment

In [4]:
# Get the FairMOT Model
! git clone https://github.com/ifzhang/FairMOT     # the FairMot scripts
! git clone https://github.com/CharlesShang/DCNv2  # package for using DLA-34 model, the backbone neural network of FairMOT

Cloning into 'FairMOT'...
remote: Enumerating objects: 719, done.
remote: Total 719 (delta 0), reused 0 (delta 0), pack-reused 719
Receiving objects: 100% (719/719), 63.34 MiB | 17.25 MiB/s, done.
Resolving deltas: 100% (383/383), done.
Cloning into 'DCNv2'...
remote: Enumerating objects: 214, done.
remote: Total 214 (delta 0), reused 0 (delta 0), pack-reused 214
Receiving objects: 100% (214/214), 1.42 MiB | 1.02 MiB/s, done.
Resolving deltas: 100% (121/121), done.


#### Ensure CUDA is enabled

In [7]:
! nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


#### Ensure the Python version is 3.8

In [8]:
! python3 --version

Python 3.7.10


#### Install PyTorch

In [9]:
! pip install torch==1.4.0 torchvision==0.5.0

  Using cached torch-1.4.0-cp37-cp37m-manylinux1_x86_64.whl (753.4 MB)
  Using cached torchvision-0.5.0-cp37-cp37m-manylinux1_x86_64.whl (4.0 MB)
  Attempting uninstall: torch
    Found existing installation: torch 1.2.0
    Uninstalling torch-1.2.0:
      Successfully uninstalled torch-1.2.0
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.4.0a0+6b959ee
    Uninstalling torchvision-0.4.0a0+6b959ee:
      Successfully uninstalled torchvision-0.4.0a0+6b959ee


#### Install CONDA

If CONDA is already installed, skip this section.

In [10]:
! conda --version

conda 4.12.0


Install Conda which is required for Fair MOT's requirements.

In [11]:
! pip install -q condacolab

import condacolab
condacolab.install()

✨🍰✨ Everything looks OK!


#### Build DCNv2 - Deformable Convolutional Networks v2 with Pytorch

Ensure GPU is enabled before running build. 

Debugging link: https://github.com/CharlesShang/DCNv2/issues/98#issuecomment-939261953

Remove the build folder if it exists

In [12]:
! rm -rf /content/build

Build DCNv2

In [13]:
! python ./DCNv2/setup.py build develop

running build
running build_py
creating build
creating build/lib.linux-x86_64-3.7
creating build/lib.linux-x86_64-3.7/DCNv2
copying DCNv2/testcuda.py -> build/lib.linux-x86_64-3.7/DCNv2
copying DCNv2/__init__.py -> build/lib.linux-x86_64-3.7/DCNv2
copying DCNv2/setup.py -> build/lib.linux-x86_64-3.7/DCNv2
copying DCNv2/testcpu.py -> build/lib.linux-x86_64-3.7/DCNv2
copying DCNv2/dcn_v2.py -> build/lib.linux-x86_64-3.7/DCNv2
running build_ext
building '_ext' extension
creating build/temp.linux-x86_64-3.7
creating build/temp.linux-x86_64-3.7/content
creating build/temp.linux-x86_64-3.7/content/DCNv2
creating build/temp.linux-x86_64-3.7/content/DCNv2/src
creating build/temp.linux-x86_64-3.7/content/DCNv2/src/cpu
creating build/temp.linux-x86_64-3.7/content/DCNv2/src/cuda
g++ -Wsign-compare -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -fPIC -DWITH_CUDA -I/content/DCNv2/src -I/usr/local/lib/python3.7/site-packages/torch/include -I/usr/local/lib/python3.7/site-packages/torch/include/tor

Run test CUDA.  

For the *UserWarning: At least one of the inputs that requires gradient is not of double precision floating point. This check will likely fail if all the inputs are not of double precision floating point. 'At least one of the inputs that requires gradient '* error:

* These are known issues on DCNv2 and are related to shortcomings related to grad-check, it is working fine for me (Even the Jacobian mismatch error is occuring).

Source: https://github.com/Mukosame/Zooming-Slow-Mo-CVPR-2020/issues/11#issuecomment-611352001

In [14]:
! python /content/DCNv2/testcuda.py

torch.Size([2, 64, 128, 128])
torch.Size([20, 32, 7, 7])
torch.Size([20, 32, 7, 7])
torch.Size([20, 32, 7, 7])
0.971507, 1.943014
0.971507, 1.943014
Zero offset passed
/usr/local/lib/python3.7/site-packages/torch/autograd/gradcheck.py:242: UserWarning: At least one of the inputs that requires gradient is not of double precision floating point. This check will likely fail if all the inputs are not of double precision floating point. 
  'At least one of the inputs that requires gradient '
check_gradient_dpooling: True
Traceback (most recent call last):
  File "/content/DCNv2/testcuda.py", line 265, in <module>
    check_gradient_dconv()
  File "/content/DCNv2/testcuda.py", line 97, in check_gradient_dconv
    eps=1e-3, atol=1e-4, rtol=1e-2))
  File "/usr/local/lib/python3.7/site-packages/torch/autograd/gradcheck.py", line 289, in gradcheck
    'numerical:%s\nanalytical:%s\n' % (i, j, n, a))
  File "/usr/local/lib/python3.7/site-packages/torch/autograd/gradcheck.py", line 227, in fail_tes

#### Install FairMOT requirements

In [15]:
! pip install -r /content/FairMOT/requirements.txt
! conda install pytorch==1.2.0 torchvision==0.4.0 cudatoolkit=10.0 -c pytorch -y
! conda install ffmpeg -y

Solving environment: / - \ | / - \ | / WARNING conda.core.solve:_add_specs(649): pinned spec cudatoolkit=11.1 conflicts with explicit specs.  Overriding pinned spec.
- \ | / - \ | / done

# All requested packages already installed.

Solving environment: \ | / - \ | / - \ | / - \ done

# All requested packages already installed.



#### Install packages for the DLA-34 Model

If these packages were missing, make sure to restart your runtime after installing.

In [16]:
! pip install boto3
! pip install opencv-python
! pip install sagemaker

#### Download pre-trained model weights of the DLA-34 backbone model

Documentation for AWS SageMaker: https://aws.amazon.com/sagemaker/



In [15]:
# import os

# os.environ['AWS_DEFAULT_REGION'] = 'ap-southeast-1'

In [16]:
# import boto3
# import cv2
# from PIL import Image
# import sagemaker

# sess = sagemaker.Session()
# s3 = boto3.client('s3')

# # S3 bucket name and directories for the pre-trained DLA34 model
# bucket = 'pedestrian-tracker'  
# s3key_model = 'raw-pretrained-model'
# fname = 'fairmot_dla34.pth'
# s3.download_file(bucket, s3key + '/' + fname,  '/content/FairMOT/model/' + fname)


In [17]:
!pip install matplotlib
!pip install ipykernel

In [18]:
! sudo apt-get update
! sudo apt-get install libssl-dev
# ! pip install -e git+https://github.com/samson-wang/cython_bbox.git#egg=cython-bbox

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:15 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bioni

#### Installing Cython BBox

1. Download the tar file from https://pypi.org/project/cython-bbox/#files

2. Make this change: extra_compile_args=['-Wno-cpp',  '-Wno-unused-function', '-std=c99'], in setup.py. (Source: https://github.com/Zhongdao/Towards-Realtime-MOT/issues/165#issuecomment-1042496502)

3. Zip up file again, "tar -cvzf cython_bbox.tar.gz cython_bbox-0.1.3".  https://www.makeuseof.com/extract-tar-gz/

4. Upload and install on Google Colab.


In [19]:
# ! sudo tar -xvf /content/cython_bbox-0.1.3.tar.gz
# ! python setup.py bdist_wheel
# ! pip install -e git+https://github.com/samson-wang/cython_bbox.git#egg=cython-bbox
# ! pip install -e /content/cython_bbox-0.1.3/dist/cython_bbox-0.1.3-py3.9-macosx-11-x86_64.egg
! pip install -e /path/cython_bbox-0.1.3.tar/dist/cython_bbox-0.1.3/cython_bbox-0.1.3
! pip install cython_bbox

ERROR: /path/cython_bbox-0.1.3.tar/dist/cython_bbox-0.1.3/cython_bbox-0.1.3 is not a valid editable requirement. It should either be a path to a local project or a VCS URL (beginning with bzr+http, bzr+https, bzr+ssh, bzr+sftp, bzr+ftp, bzr+lp, bzr+file, git+http, git+https, git+ssh, git+git, git+file, hg+file, hg+http, hg+https, hg+ssh, hg+static-http, svn+ssh, svn+http, svn+https, svn+svn, svn+file).


In [20]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# ! pip install motmetrics
! pip install dcnv2

In [22]:
!python /content/FairMOT/src/demo.py mot --load_model /content/FairMOT/src/lib/models/model.py \
        --conf_thres 0.3 --det_thres 0.3 --nms_thres 0.4 --track_buffer 30 \
        --input-video /content/FairMOT/videos/MOT16-03.mp4 \
        --output-root /content/FairMOT/output

Traceback (most recent call last):
  File "/content/FairMOT/src/demo.py", line 14, in <module>
    from track import eval_seq
  File "/content/FairMOT/src/track.py", line 15, in <module>
    from tracker.multitracker import JDETracker
  File "/content/FairMOT/src/lib/tracker/multitracker.py", line 13, in <module>
    from models.model import create_model, load_model
  File "/content/FairMOT/src/lib/models/model.py", line 11, in <module>
    from .networks.pose_dla_dcn import get_pose_net as get_dla_dcn
  File "/content/FairMOT/src/lib/models/networks/pose_dla_dcn.py", line 16, in <module>
    from dcn_v2 import DCN
ModuleNotFoundError: No module named 'dcn_v2'
